In [4]:
import boto3
from botocore.exceptions import ClientError

# Initialize DynamoDB resource
dynamodb = boto3.resource('dynamodb', region_name='us-east-1')  # Change region

# Reference your table
table = dynamodb.Table('nova-ingest-ads-queue')
col_name = "data"

try:
    # Scan the table but only return the specific column (attribute)
    response = table.scan(
        ProjectionExpression=col_name  # Replace with your attribute
    )

    # Extract values
    items = response.get('Items', [])
    values = [item.get(col_name) for item in items]

    print("Fetched values:", values)

    # Handle pagination (DynamoDB Scan can return partial results)
    while 'LastEvaluatedKey' in response:
        response = table.scan(
            ProjectionExpression=col_name,
            ExclusiveStartKey=response['LastEvaluatedKey']
        )
        items = response.get('Items', [])
        values.extend(item.get(col_name) for item in items)

    print("All values:", values)

except ClientError as e:
    print(f"Error scanning table: {e.response['Error']['Message']}")


Error scanning table: Invalid ProjectionExpression: Attribute name is a reserved keyword; reserved keyword: data


In [7]:
from datetime import date, datetime, timezone

x = datetime.now(timezone.utc).isoformat()

In [8]:
type(x)

str

In [ ]:
import re

ADS_BIBCODE_RE = re.compile(r"^\S{4}\S*$")  # (placeholder) keep loose unless you want strict ADS format


# bibcode = "2022yCat..22570049C"
bibcode = "2021ITNS...815C" 
def _clean_bibcode(cls, v):
    if v is None:
        raise ValueError("bibcode is required.")
    s = str(v).strip()
    if not s:
        raise ValueError("bibcode cannot be empty or whitespace.")
    # keep pattern loose unless you want to enforce the exact ADS bibcode grammar
    if not ADS_BIBCODE_RE.match(s):
        print("bibcode %r did not match ADS_BIBCODE_RE; keeping as-is", s)
        # logger.info("bibcode %r did not match ADS_BIBCODE_RE; keeping as-is", s)
    return s

_clean_bibcode(None, bibcode)

bibcode %r did not match ADS_BIBCODE_RE; keeping as-is 2021ITNS...68.. 815C


'2021ITNS...68.. 815C'